In [121]:
import pickle
# import import_ipynb
from XGBoost import pre_process
import numpy as np
from sklearn.preprocessing import LabelBinarizer, StandardScaler
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV, train_test_split
import pandas as pd
from sklearn.metrics import *
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import confusion_matrix

In [122]:
file = open('pickle/model.pkl','rb')
added = pickle.load(file)
    

In [123]:
added

"['France_lbinarizer', 'Germany_lbinarizer', 'Female_lbinarizer', 'Age_SScaler', 'Tenure_SScaler', 'Balance_SScaler', 'HasCrCard_SScaler', 'EstimatedSalary_SScaler']"

In [124]:
geog_lb = pickle.load(file)
# gender_lb = pickle.load(file)
# female_lb =  pickle.load(file)
# age_ss = pickle.load(file)
# tenure_ss = pickle.load(file)
# bal_ss = pickle.load(file)
# has_crd_ss = pickle.load(file)
# sal_ss=pickle.load(file)
# classifier = pickle.load(file)

In [125]:
geog_lb

[LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 StandardScaler(copy=True, with_mean=True, with_std=True),
 StandardScaler(copy=True, with_mean=True, with_std=True),
 StandardScaler(copy=True, with_mean=True, with_std=True),
 StandardScaler(copy=True, with_mean=True, with_std=True),
 StandardScaler(copy=True, with_mean=True, with_std=True),
 XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
        colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
        max_delta_step=0, max_depth=3, min_child_weight=1, missing=nan,
        n_estimators=100, n_jobs=1, nthread=None,
        objective='binary:logistic', random_state=0, reg_alpha=0,
        reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
        subsample=1, verbosity=1)]

In [126]:
lb_Geo = geog_lb[0]
lb_gen = geog_lb[1]
age_ss = geog_lb[2]
tenure_ss = geog_lb[3]
bal_ss = geog_lb[4]
has_crd_ss = geog_lb[5]
sal_ss= geog_lb[6]
classifier = geog_lb[7]

In [127]:
test = pd.read_csv('Data/test.csv')

In [128]:
X = test[['Age', 'Tenure', 'Geography','Gender','Balance', 'HasCrCard', 'EstimatedSalary']]
y = test['Exited']

In [129]:
X.head()

,Age,Tenure,Geography,Gender,Balance,HasCrCard,EstimatedSalary
0,35,8,Germany,Female,131101.04,1,192852.67
1,40,2,France,Female,102967.41,1,128702.10
2,42,8,Spain,Female,95386.82,1,75732.25
3,32,4,France,Male,112079.58,0,89368.59
4,38,5,Germany,Male,163034.82,1,135662.17


In [130]:
cols=['Geography','Gender']
added_cols =[]
def binarizer(X,col,lb):
    sorted_col = np.sort(X[col].unique())
    binarized = lb.transform(X[col])
    for i in range(0,len(sorted_col)-1):
        X[sorted_col[i]]=binarized[:,i]
        added_cols.append(sorted_col[i])
    X.drop([col],axis=1,inplace=True)
    return X

In [131]:
X = binarizer(X,cols[0],lb_Geo)
X = binarizer(X,cols[1],lb_gen)

/home/admin1/anaconda3/envs/prayas_env/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/admin1/anaconda3/envs/prayas_env/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/admin1/anaconda3/envs/prayas_env/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata

In [135]:
def scaler(X, col, ss):
    if col not in added_cols:
        X[col] = ss.transform(np.array(X[col]).reshape(-1,1))

In [136]:
added

"['France_lbinarizer', 'Germany_lbinarizer', 'Female_lbinarizer', 'Age_SScaler', 'Tenure_SScaler', 'Balance_SScaler', 'HasCrCard_SScaler', 'EstimatedSalary_SScaler']"

In [137]:
scaler(X,'Age',age_ss)
scaler(X,'Tenure',tenure_ss)
scaler(X,'Balance',bal_ss)
scaler(X,'HasCrCard',has_crd_ss)
scaler(X,'EstimatedSalary',sal_ss)

/home/admin1/anaconda3/envs/prayas_env/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/admin1/anaconda3/envs/prayas_env/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/admin1/anaconda3/envs/prayas_env/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/admin1/anaconda3/envs/prayas_env/lib/python3.7/site-packages/ipykernel_

In [138]:
X.head()

,Age,Tenure,Balance,HasCrCard,EstimatedSalary,France,Germany,Female
0,-0.370117,1.036965,0.875639,0.645642,1.613182,0,1,0
1,0.107278,-1.034856,0.424902,0.645642,0.496813,1,0,0
2,0.298236,1.036965,0.303451,0.645642,-0.424985,0,0,0
3,-0.656554,-0.344249,0.570890,-1.548847,-0.187681,1,0,1
4,-0.083680,0.001054,1.387259,0.645642,0.617935,0,1,1


In [139]:
pred = classifier.predict(X)

In [140]:
confusion_matrix(y,pred)

array([[392,   9],
       [ 62,  37]])